In [1]:
import osmnx as ox
import pandas as pd
import time
import requests
import os
from pathlib import Path
from datetime import datetime

 # 1. Get list of all US Counties 
 
The attempt to download data for a very large county - Los Angeles - was successful.
 
Now, i will attempt to obtain a list of all US Counties...

I should be able to get this data from the US Census API Geography Endpoint: https://www.census.gov/data/developers/data-sets/geo-info.html

In [2]:
census_api_key = " 349179f9505abb551eb4de4ab998d2a0f58d06aa"

In [3]:
url = 'https://api.census.gov/data/2023/geoinfo'

# needed! 
headers = {'User-Agent': 'Mozilla/5.0'}

params = {'get':'NAME', 
                        'for': 'county:*',
                      }

response = requests.get(url, headers=headers, params=params, timeout=10)

if response.status_code == 200:
    data = response.json()
    print("Success")
else:
    print(response.status_code)

Success


Convert to a Pandas dataframe

In [4]:
# extract county names
counties_list = [item[0] for item in data][1:]

counties_df = pd.DataFrame(counties_list, columns=['name_full'])

# add State column 
counties_df['state'] = counties_df.apply(lambda x: x['name_full'].split(",")[1].strip(), axis=1)
# add Counties
counties_df['county'] = counties_df.apply(lambda x: x['name_full'].split(",")[0].strip(), axis=1)

counties_df.head()

,name_full,state,county
0,"Autauga County, Alabama",Alabama,Autauga County
1,"Baldwin County, Alabama",Alabama,Baldwin County
2,"Barbour County, Alabama",Alabama,Barbour County
3,"Bibb County, Alabama",Alabama,Bibb County
4,"Blount County, Alabama",Alabama,Blount County


### Save the County names to file

In [5]:
counties_df.to_csv('us_counties.csv', index = False)

# 2. Download all US Counies

In [6]:
counties_df = pd.read_csv('us_counties.csv')

# df for results
results_df = pd.DataFrame(columns = ['state', 'name_full', 'name_abbrev', 'downloaded', 'file_path', 'size', 'date', 'area'])


In [7]:
counties_df.head()

,name_full,state,county
0,"Autauga County, Alabama",Alabama,Autauga County
1,"Baldwin County, Alabama",Alabama,Baldwin County
2,"Barbour County, Alabama",Alabama,Barbour County
3,"Bibb County, Alabama",Alabama,Bibb County
4,"Blount County, Alabama",Alabama,Blount County


In [8]:
def get_abbrev_name(county_row):
    county = county_row['name_full'].split(",")[0]

    # remove 'County' suffix
    suffix_loc = county.find("County")

    if suffix_loc != -1:
        county = county[:suffix_loc].strip()
    
    return county

In [ ]:
import geopandas as gpd
import osmnx as ox
import pandas as pd
from datetime import datetime

tags = {"leisure": ["pitch", "park"]}
equal_area_crs = "EPSG:5070"  # US Albers
today = datetime.today().strftime("%Y-%m-%d")

rows = []

for full_name in counties_df["name_full"].unique().tolist():
    try:
        gdf = ox.features.features_from_place(full_name, tags)

        if gdf is None or gdf.empty:
            rows.append({
                "name_full": full_name,
                "state": full_name.split(",")[-1].strip(),
                "date": today,
                "area_m2": 0.0
            })
            print(f"{full_name}: no parks found.")
            continue

        # Fix and project once
        gdf = gdf[~gdf.geometry.is_empty].copy()
        gdf["geometry"] = gdf.geometry.buffer(0)
        gdf = gdf.set_crs("EPSG:4326", allow_override=True).to_crs(equal_area_crs)

        # Compute total area (sum of polygons)
        area_m2 = gdf.geometry.area.sum()

        rows.append({
            "name_full": full_name,
            "state": full_name.split(",")[-1].strip(),
            "date": today,
            "area_m2": area_m2
        })
        print(f"{full_name}: area_m2 = {area_m2:.1f} ({len(gdf)} features)")

    except Exception as e:
        rows.append({
            "name_full": full_name,
            "state": full_name.split(",")[-1].strip(),
            "date": today,
            "area_m2": float("nan")
        })
        print(f"Error fetching {full_name}: {e}")

area_df = pd.DataFrame(rows)
print(area_df.sort_values("area_m2", ascending=False).head())


Autauga County, Alabama: area_m2 = 1335796.2 (78 features)
Baldwin County, Alabama: area_m2 = 7982886.5 (740 features)
Barbour County, Alabama: area_m2 = 7348060.2 (31 features)
Bibb County, Alabama: area_m2 = 6592400.5 (44 features)
Blount County, Alabama: area_m2 = 5974193.5 (134 features)
Bullock County, Alabama: area_m2 = 6345183.9 (8 features)
Butler County, Alabama: area_m2 = 811866.6 (61 features)
Calhoun County, Alabama: area_m2 = 3530768.0 (273 features)
Chambers County, Alabama: area_m2 = 4308940.3 (66 features)
Cherokee County, Alabama: area_m2 = 112219.2 (35 features)
Chilton County, Alabama: area_m2 = 2465167.2 (54 features)
Choctaw County, Alabama: area_m2 = 2007919.1 (28 features)
Clarke County, Alabama: area_m2 = 617753.8 (49 features)
Clay County, Alabama: area_m2 = 2286766.3 (8 features)
Cleburne County, Alabama: area_m2 = 29692.2 (14 features)
Coffee County, Alabama: area_m2 = 1447108.0 (93 features)
Colbert County, Alabama: area_m2 = 791815.4 (104 features)
Conecuh 

/opt/anaconda3/lib/python3.12/site-packages/osmnx/_overpass.py:271: UserWarning: This area is 28 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)


Aleutians East Borough, Alaska: area_m2 = 22765.5 (11 features)
Error fetching Aleutians West Census Area, Alaska: Nominatim did not geocode query 'Aleutians West Census Area, Alaska' to a geometry of type (Multi)Polygon.
Anchorage Municipality, Alaska: area_m2 = 337577.8 (4 features)
Error fetching Bethel Census Area, Alaska: Nominatim did not geocode query 'Bethel Census Area, Alaska' to a geometry of type (Multi)Polygon.
Bristol Bay Borough, Alaska: area_m2 = 116455.4 (3 features)
Error fetching Chugach Census Area, Alaska: Nominatim geocoder returned 0 results for query 'Chugach Census Area, Alaska'.
Error fetching Copper River Census Area, Alaska: Nominatim geocoder returned 0 results for query 'Copper River Census Area, Alaska'.


/opt/anaconda3/lib/python3.12/site-packages/osmnx/_overpass.py:271: UserWarning: This area is 15 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)


Denali Borough, Alaska: area_m2 = 24100.8 (7 features)
Error fetching Dillingham Census Area, Alaska: Nominatim geocoder returned 0 results for query 'Dillingham Census Area, Alaska'.
Fairbanks North Star Borough, Alaska: area_m2 = 7633966.8 (270 features)
Haines Borough, Alaska: area_m2 = 194724.4 (14 features)
Error fetching Hoonah-Angoon Census Area, Alaska: Nominatim did not geocode query 'Hoonah-Angoon Census Area, Alaska' to a geometry of type (Multi)Polygon.
Error fetching Juneau City and Borough, Alaska: No matching features. Check query location, tags, and log.


...

Elko County, Nevada: area_m2 = 2903109.0 (128 features)
Eureka County, Nevada: area_m2 = 92022.9 (23 features)
Humboldt County, Nevada: area_m2 = 472499.2 (70 features)
Lander County, Nevada: area_m2 = 372026.7 (41 features)


/opt/anaconda3/lib/python3.12/site-packages/osmnx/_overpass.py:271: UserWarning: This area is 12 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)


Lincoln County, Nevada: area_m2 = 377576.6 (49 features)
Lyon County, Nevada: area_m2 = 86425140.2 (76 features)
Mineral County, Nevada: area_m2 = 117879.6 (13 features)


/opt/anaconda3/lib/python3.12/site-packages/osmnx/_overpass.py:271: UserWarning: This area is 24 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)


Nye County, Nevada: area_m2 = 5863841.9 (113 features)
Pershing County, Nevada: area_m2 = 97433.0 (21 features)
Storey County, Nevada: area_m2 = 25428.3 (7 features)
Washoe County, Nevada: area_m2 = 21660691.9 (976 features)
White Pine County, Nevada: area_m2 = 282980.8 (43 features)
Carson City, Nevada: area_m2 = 5619739.9 (117 features)
Belknap County, New Hampshire: area_m2 = 11247770.3 (273 features)
Carroll County, New Hampshire: area_m2 = 19481994.7 (360 features)
Cheshire County, New Hampshire: area_m2 = 26650743.5 (199 features)
Coos County, New Hampshire: area_m2 = 13141639.8 (163 features)
Grafton County, New Hampshire: area_m2 = 39615893.1 (540 features)
Hillsborough County, New Hampshire: area_m2 = 22826597.1 (1159 features)
Merrimack County, New Hampshire: area_m2 = 25383287.4 (585 features)
Rockingham County, New Hampshire: area_m2 = 15059050.6 (1019 features)
Strafford County, New Hampshire: area_m2 = 5504307.6 (356 features)
Sullivan County, New Hampshire: area_m2 = 361

In [12]:
all_df = pd.read_csv('output/county_parks_all.csv')

In [15]:
all_df.parkserve_area_m2.sum()

9139511370.030994

In [18]:
all_df.combined_area_m2.sum()

14192501902.64352

In [19]:
all_df.describe()

,geoid,statefp,countyfp,osm_area_m2,parkserve_area_m2,combined_area_m2,overlap_area_m2
count,3235.000000,3235.000000,3235.000000,3.235000e+03,3.235000e+03,3.235000e+03,3.235000e+03
mean,31522.781453,31.419784,102.997836,3.003121e+06,2.825197e+06,4.387172e+06,1.441145e+06
std,16431.824116,16.417990,106.430682,8.930061e+06,1.235016e+07,1.556662e+07,5.289447e+06
min,1001.000000,1.000000,1.000000,0.000000e+00,0.000000e+00,0.000000e+00,-9.313226e-10
25%,19036.000000,19.000000,35.000000,1.359424e+05,0.000000e+00,1.651187e+05,0.000000e+00
50%,30035.000000,30.000000,79.000000,5.252738e+05,2.486686e+05,6.505762e+05,1.474231e+05
75%,46126.000000,46.000000,133.000000,1.956550e+06,1.157399e+06,2.405599e+06,6.877976e+05
max,78030.000000,78.000000,840.000000,1.653765e+08,3.086142e+08,3.588217e+08,9.206724e+07
